<a href="https://colab.research.google.com/github/sajiaakhter/mec-mini-projects/blob/master/FutureSalesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download Kaggle Data in Colab**

In [1]:
! pip install -q kaggle


In [ ]:
#before running copy kaggle.json using current APi
! mkdir ~/.kaggle
! cp /kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! ls -al /root/.kaggle/
!kaggle datasets list

**Downloading Future Sales data**

In [7]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

 93% 14.0M/15.1M [00:01<00:00, 19.0MB/s]
100% 15.1M/15.1M [00:01<00:00, 11.9MB/s]


In [8]:
#Unzip
! mkdir /content/futureSales
! unzip /content/competitive-data-science-predict-future-sales.zip -d /content/futureSales 

Archive:  /content/competitive-data-science-predict-future-sales.zip
  inflating: /content/futureSales/item_categories.csv  
  inflating: /content/futureSales/items.csv  
  inflating: /content/futureSales/sales_train.csv  
  inflating: /content/futureSales/sample_submission.csv  
  inflating: /content/futureSales/shops.csv  
  inflating: /content/futureSales/test.csv  


**Exploratory data analysis**

In [9]:
#import libraries
import numpy as np
import pandas as pd

In [11]:
sales_train = pd.read_csv('/content/futureSales/sales_train.csv') 
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [14]:
items = pd.read_csv('/content/futureSales/items.csv') 
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [15]:
item_categories = pd.read_csv('/content/futureSales/item_categories.csv')
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [16]:
shops = pd.read_csv('/content/futureSales/shops.csv')
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
